# Implementacion del algoritmo CLR

In [1]:
# Dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoCV
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [2]:
# Cargue de los datos | informacion
df = pd.read_excel("../Archivos Generados/DatasetFinal.xlsx")

In [3]:
# Se muestra los primeros 5 elementos
df.head(5)

,ID_LOTE,TIPO_SIEMBRA,SEM_TRATADAS,MATERIAL_GENETICO,CULT_ANT,DRENAJE,METODO_COSECHA,ALMACENAMIENTO_FINCA,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,...,Temp_Max_Avg_Mad,Temp_Min_Avg_Mad,Temp_Avg_Mad,Diurnal_Range_Avg_Mad,Sol_Ener_Accu_Mad,Temp_Max_34_Freq_Mad,Rain_Accu_Mad,Rain_10_Freq_Mad,Rhum_Avg_Mad,RDT_AJUSTADO
0,40,Mecanizado,NO,PIONEER 30F32,Algodon,SI,Manual,NO,5,63,...,32.05,23.60,27.83,8.45,13197.57,0.05,279.3,0.23,82.41,4767.44
1,43,Mecanizado,SI,DK 234,Maiz,SI,Manual,NO,5,64,...,32.37,23.49,27.93,8.89,12436.49,0.03,221.2,0.26,81.86,4651.16
2,44,Mecanizado,NO,PIONEER 30F32,Algodon,SI,Manual,NO,5,59,...,32.17,23.53,27.85,8.63,11267.17,0.03,226.0,0.27,82.61,5180.23
3,45,Mecanizado,NO,Otro,Algodon,SI,Manual,NO,5,64,...,32.19,23.54,27.86,8.65,11066.68,0.03,223.2,0.29,81.84,4897.67
4,46,Mecanizado,NO,Otro,Algodon,SI,Manual,NO,5,63,...,32.19,23.54,27.86,8.65,11066.68,0.03,223.2,0.29,81.84,5302.33


In [4]:
# Selección de las variables categoricas
cat_features = df.select_dtypes(include = ["object", "category"]).columns
cat_features

Index(['TIPO_SIEMBRA', 'SEM_TRATADAS', 'MATERIAL_GENETICO', 'CULT_ANT',
       'DRENAJE', 'METODO_COSECHA', 'ALMACENAMIENTO_FINCA',
       'TERRENO_CIRCUN_RASTA', 'POSICION_PERFIL_RASTA', 'PEDREG_PERFIL_ROCAS',
       'CAP_ENDURE_RASTA', 'MOTEADOS_RASTA', 'MOTEADOS_MAS70cm._RASTA',
       'ESTRUCTURA_RASTA', 'OBSERVA_EROSION_RASTA', 'OBSERVA_MOHO_RASTA',
       'OBSERVA_COSTRAS_DURAS_RASTA', 'SITIO_EXPUESTO_SOL_RASTA',
       'OBSERVA_COSTRAS_BLANCAS_RASTA', 'OBSERVA_COSTAS_NEGRAS_RASTA',
       'REGION_SECA_ARIDA_RASTA', 'OBSERVA_RAICES_VIVAS_RASTA',
       'OBSERVA_PLANTAS_PEQUENAS_RASTA', 'OBSERVA_HOJARASCA_MO_RASTA',
       'SUELO_NEGRO_BLANDO_RASTA', 'CUCHILLO_PRIMER_HTE_RASTA',
       'CERCA_RIOS_QUEBRADAS_RASTA', 'RECUBRIMIENTO_VEGETAL__SUELO_RASTA',
       'd.interno', 'drenaje_externo'],
      dtype='object')

In [5]:
# Tipos de datos de las variables
df.dtypes

ID_LOTE                   int64
TIPO_SIEMBRA             object
SEM_TRATADAS             object
MATERIAL_GENETICO        object
CULT_ANT                 object
                         ...   
Temp_Max_34_Freq_Mad    float64
Rain_Accu_Mad           float64
Rain_10_Freq_Mad        float64
Rhum_Avg_Mad            float64
RDT_AJUSTADO            float64
Length: 116, dtype: object

In [6]:
# Longitud de las variables categoricas
len(cat_features)

30

In [7]:
# Codificación dummy de las variables categoricas
df_cat = pd.get_dummies(df[cat_features], drop_first = True, dummy_na = True)
df_cat.head()

,TIPO_SIEMBRA_Mecanizado,TIPO_SIEMBRA_nan,SEM_TRATADAS_SI,SEM_TRATADAS_nan,MATERIAL_GENETICO_ADV 9339 (Syngenta),MATERIAL_GENETICO_CORPOICA V 114,MATERIAL_GENETICO_Cerato (Syngenta),MATERIAL_GENETICO_DK 1040,MATERIAL_GENETICO_DK 1596,MATERIAL_GENETICO_DK 234,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [8]:
# Se eliminan las variables categoricas  del dataframe principal y se mantienen unicamente las variables numericas
df.drop(cat_features, axis = 1, inplace = True)

In [9]:
df.columns

Index(['ID_LOTE', 'DIAS_EN_EMERGER', 'DIAS_EN_EMERGER_A_FLORECER',
       'DIAS_EN_FLORECER_A_COSECHAR', 'POBLACION_20DIAS_AJT', 'ALTURA_LOT',
       'ContEnfQui_Emer_Flor', 'ContEnfQui_Flor_Cose', 'ContMalMec_Siem_Emer',
       'ContMalMec_Emer_Flor', 'ContMalMec_Flor_Cose', 'ContMalQui_Antes_Siem',
       'ContMalQui_Siem_Emer', 'ContMalQui_Emer_Flor', 'ContMalQui_Flor_Cose',
       'ContPlaQui_Antes_Siem', 'ContPlaQui_Siem_Emer', 'ContPlaQui_Emer_Flor',
       'ContPlaQui_Flor_Cose', 'TotN_Antes_Siem', 'TotN_Siem_Emer',
       'TotN_Emer_Flor', 'TotP_Antes_Siem', 'TotP_Siem_Emer', 'TotP_Emer_Flor',
       'TotK_Antes_Siem', 'TotK_Siem_Emer', 'TotK_Emer_Flor',
       'FerOrg_Emer_Flor', 'FerQui_Antes_Siem', 'FerQui_Siem_Emer',
       'FerQui_Emer_Flor', 'PENDIENTE_RASTA', 'NO_CAPAS_RASTA', 'PH_RASTA',
       'PROFUND_CAP_ENDURE_RASTA', 'ESPESOR_CAP_ENDURE_RASTA',
       'PROFUND_MOTEADOS_RASTA', 'PROFUND_RAICES_VIVAS_RASTA', 'prof_efectiva',
       'Porc_A', 'Porc_Ar', 'Porc_ArA', 'P

In [10]:
# Se construye un unico dataset | vista minable con valores numericos
df_final = pd.concat([df,df_cat], axis = 1)

In [11]:
df_final

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
0,40,5,63,68,60000,13,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,43,5,64,63,60000,15,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,44,5,59,66,60000,12,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,45,5,64,59,60000,12,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,46,5,63,60,60000,16,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,4378,7,47,84,70000,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,4380,6,49,79,61000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
797,4382,7,48,95,65000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Columns: 189 entries, ID_LOTE to drenaje_externo_nan
dtypes: float64(55), int64(31), uint8(103)
memory usage: 617.3 KB


# Modelado
## Regresion Lineal (Lasso)

In [13]:
# Division de los datos | variables dependientes
# X: Dependiente
# Y: Independiente
Y = df_final.RDT_AJUSTADO
X = df_final.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1)

In [14]:
Y

0      4767.44
1      4651.16
2      5180.23
3      4897.67
4      5302.33
        ...   
794    6418.60
795    5581.40
796    5106.98
797    5764.19
798    7341.86
Name: RDT_AJUSTADO, Length: 799, dtype: float64

In [15]:
# Se deviden los datos en conjunto de entranmiento y test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=40)
print(X_train.shape); print(X_test.shape)


(639, 187)
(160, 187)


In [16]:
# Implementación regresion lineal simple
lr = LinearRegression()
lr.fit(X_train, y_train)
# Obtencion de la metadata del modelo
lr.get_params(deep=True)

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [17]:
# Calculo del error 
pred_train_lr= lr.predict(X_train)
print("======================= Conjunto de entrenamiento ==========================")
print((mean_squared_error(y_train,pred_train_lr,squared=False)))
print(r2_score(y_train, pred_train_lr))
print("======================== Conjuneto de test  =================")
pred_test_lr= lr.predict(X_test)
print((mean_squared_error(y_test,pred_test_lr,squared=False))) 
print(r2_score(y_test, pred_test_lr))

======================= Conjunto de entrenamiento ==========================
648.3603502822376
0.780332724877175
======================== Conjuneto de test  =================
980.7553681822333
0.504322154691877


# Implementacion del regresor lineal lasso

-  Lasso internamente realiza selecccion de atributos dejando unicamente los atributos mas importantes
- Lasso es particularmente adecuado para situaciones en las que la cantidad de características es mayor que la cantidad de observaciones
- Para el caso del enfoque CLR es conveninte trabajar con algoritmos que no realizan seleccion de atributos
- Regresion lineal lasso es un tipo de regresion lineal regularizada.
- Los coeficientes del modelo se encuentran mediante un proceso de optimización que busca minimizar el error de suma cuadrada entre las predicciones ( yhat ) y los valores objetivo esperados ( y ). loss = sum i=0 to n (y_i – yhat_i)^2
- Una penalización L1 minimiza el tamaño de todos los coeficientes y permite que cualquier coeficiente alcance el valor cero, eliminando efectivamente las características de entrada del modelo
- Se utiliza un hiperparámetro llamado “ lambda ” que controla la ponderación de la penalización a la función de pérdida. Un valor predeterminado de 1,0 dará ponderación total a la penalización; un valor de 0 excluye la penalización. [entre 0,1]


**Observación** teniendo en cuenta que el algoritmo lasso realiza selección de atribributos, se realizan las pruebas pertinentes

In [18]:

# Se define el modelo
model = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print("Vector MAE , en c/d iteración de la validación cruzada: ", scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Vector MAE , en c/d iteración de la validación cruzada:  [768.15621601 631.30273921 658.1481118  555.67760894 607.37863181
 671.12695857 768.12318003 647.65525441 667.38864302 610.45549141
 605.81682776 626.04287112 642.37740421 613.4548015  619.78440439
 751.44305232 729.58221554 749.34338298 666.26715831 655.67321992
 721.87616213 629.02759196 669.7876704  737.3529658  719.08543267
 628.75246278 665.38456555 557.30811746 633.45934383 689.39689639]
Mean MAE: 663.221 (56.935)


In [19]:
# Teniendo en cuenta el MAE se ajusta un modelo bajo esta regresión
# Se define el modelo
modelLasso = Lasso(alpha=0.78)
# fit model
modelLasso.fit(X_train, y_train)
# make a prediction
yhat = modelLasso.predict(X_test)
# summarize prediction
print('Predicted:', yhat)


Predicted: [3864.40142212 4504.12802402 5891.78104213 6186.3064082  3603.32705091
 3234.68049845 4195.85899911 6326.0813635  3695.51329718 3994.4484464
 4693.01333368 6998.92201556 4750.77377086 2579.07561772 4448.92845453
 3628.33157008 3749.20482711 3536.11619878 2510.14480984 3689.96295039
 5301.9037827  4873.04261933 3844.62128871 2693.17110539 6637.88143723
 6162.95297403 4913.35227526 5525.56178316 4462.83943879 6108.33368758
 2026.25616093 4044.27546445 4400.01357822 1773.25286302 5802.57182953
 5832.18047603 4816.35330463 4505.52885445 4367.7241921  4618.1295678
 4646.79783898 4829.42335804 4282.96785365 5295.05635071 3027.08727354
 3646.39096111 5288.66453163 3917.21852331 5743.14046387 6337.06859261
 4642.75133129 4208.71111419 4873.41318673 4531.64608273 2799.19506066
 5909.90883165 5301.51374555 4678.22723521 4753.2033525  5143.56215975
 5991.63328828 4709.76030543 4232.90104613 5650.66262048 6673.19092086
 4361.70527284 3995.81409061 6090.99833446 4842.1606411  6532.829688

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.213e+07, tolerance: 1.223e+05
  model = cd_fast.enet_coordinate_descent(


In [20]:
# Coeficiente de determinacion de la predicción
R_2 = modelLasso.score(X_train, y_train)
print("El ajuste del modelo (Training ) es : ", R_2)
# Se calcula el error deel modelo
# Calculo del error 
print("======================= Conjunto de test ==========================")
print("MSE: ", (mean_squared_error(y_test,yhat,squared=False)))
print("MAE", mean_absolute_error(y_test,yhat))
print("r**2: ", r2_score(y_test, yhat))

El ajuste del modelo (Training ) es :  0.768140875884345
======================= Conjunto de test ==========================
MSE:  923.2894310846524
MAE 679.9351130438965
r**2:  0.5607074391781751


In [21]:
# Caracteristicas del modelo ajustado con Regresion Lineal Lasso
# Coeficientes del modelo ajustado..
modelLasso.coef_
# intercepto de la funcion de desicion
modelLasso.intercept_
# Cantidad de caracteristicas seleccionadas (Variables independientes)
modelLasso.n_features_in_
# Nombre de los atributos | caracteristicas seleccionadas
modelLasso.feature_names_in_


array(['DIAS_EN_EMERGER', 'DIAS_EN_EMERGER_A_FLORECER',
       'DIAS_EN_FLORECER_A_COSECHAR', 'POBLACION_20DIAS_AJT',
       'ALTURA_LOT', 'ContEnfQui_Emer_Flor', 'ContEnfQui_Flor_Cose',
       'ContMalMec_Siem_Emer', 'ContMalMec_Emer_Flor',
       'ContMalMec_Flor_Cose', 'ContMalQui_Antes_Siem',
       'ContMalQui_Siem_Emer', 'ContMalQui_Emer_Flor',
       'ContMalQui_Flor_Cose', 'ContPlaQui_Antes_Siem',
       'ContPlaQui_Siem_Emer', 'ContPlaQui_Emer_Flor',
       'ContPlaQui_Flor_Cose', 'TotN_Antes_Siem', 'TotN_Siem_Emer',
       'TotN_Emer_Flor', 'TotP_Antes_Siem', 'TotP_Siem_Emer',
       'TotP_Emer_Flor', 'TotK_Antes_Siem', 'TotK_Siem_Emer',
       'TotK_Emer_Flor', 'FerOrg_Emer_Flor', 'FerQui_Antes_Siem',
       'FerQui_Siem_Emer', 'FerQui_Emer_Flor', 'PENDIENTE_RASTA',
       'NO_CAPAS_RASTA', 'PH_RASTA', 'PROFUND_CAP_ENDURE_RASTA',
       'ESPESOR_CAP_ENDURE_RASTA', 'PROFUND_MOTEADOS_RASTA',
       'PROFUND_RAICES_VIVAS_RASTA', 'prof_efectiva', 'Porc_A', 'Porc_Ar',
       'Por

### Ajuste de Hiperparametros (alpha) | Obtención del mejor modelo

In [22]:
# Dependencias adicionales
import sklearn
from numpy import arange
from sklearn.linear_model import LassoCV

# Se define el metodo de evaluación del modelo (validacion cruzada)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# Se define el modelo
modelCv = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# fit model
modelCv.fit(X, Y)


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10718441.186295807, tolerance: 135424.0452085728
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21341033.437585205, tolerance: 135797.19253151235
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 57209419.134290546, tolerance: 139290.11783302567
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 393878.71776920557, tolerance: 139290.11783302567
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 162103.8389145136, tolerance: 135797.19253151235
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 154537.15461051464, tolerance: 140728.0293495098
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\P

LassoCV(alphas=array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99]),
        cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1), n_jobs=-1)

In [23]:
# Propiedades del modelo
# Coeficientes del modelo
modelCv.coef_
# Valor de alfa seleccionado
print("Valor de alfa: ", modelCv.alpha_)
print("Version de Sklearn: ", sklearn.__version__)
#  Nombre de las caracteristicas seleccionadas 
modelCv.feature_names_in_
# Numero de caracteristicas seleccionadas
print("Numero de caracteristicas Seleccionadas: ", modelCv.n_features_in_)


Valor de alfa:  0.78
Version de Sklearn:  1.2.2
Numero de caracteristicas Seleccionadas:  187


### Conclusiones
- El regresor lineal lasso realiza internamente seleccion de atributos mediante un parametro de configuracion denominado alfa - [tasa de regularización], para este caso especifico despues de realizar la codificacion de los datos a formato dummy y pasar lso datos por el regresor se observa que se conservan las 187 caracteristicas (despues de la dumificación).
- El mejor valor de MSE obteniendo despues de encontrar  [alfa = 0.78] con la validación cruzada es de 680 logrando un ajuste unicamnete del 56% lo cual indica que es un modelo extremadamente malo.
- Se va a realizar las pruebas con el regresor Ridge y compara los respectivos resultados.


# Implementación del regreson Lineal Ridge
- Ridge es una regresión lineal regularizada que incluye una penalización L2.
- Los coeficientes de cada modelos de regresion lineal se encuentran mediante un proceso de optimización cuyo objetivo es minimizar el error de las predicciones yhat
- La caracteristica principal del regresor ridge es que agrega una penalizacion L2 para minimizar la suma de los coeficientes al cuadrado.
- Una penalización L2 minimiza el tamaño de todos los coeficientes, aunque evita que se eliminen del modelo al permitir que su valor se convierta en cero.
- La penalización de ridgue s maneja con un parametro de configuracion denominado (lamda) un valor predeterminado de 1,0 ponderará completamente la penalización; un valor de 0 excluye la penalización

In [24]:
# Dependencias necesarias
from sklearn.linear_model import Ridge


# Se define el modelo
model = Ridge(alpha=1.0)
# Se define el metodo de evaluación
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# Se evalua el modelo mediante la metrica MAE
scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
print("Vector de MAE: ", scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Vector de MAE:  [762.15140952 629.65478415 652.23632599 551.82672659 606.89283597
 673.96531923 770.03288884 636.90922474 661.64687451 598.09537759
 599.04874548 613.61085775 643.86601428 611.53087344 623.33273837
 744.45346341 722.81721908 746.43795707 640.92093595 655.24512255
 712.2878307  627.91020937 668.29897561 737.10487304 715.70390307
 627.82688349 663.8666971  556.78827648 622.92348923 680.15185942]
Mean MAE: 658.585 (57.058)


**Nota**  De manera analoga al caso anterior, se utiliza validacion cruzada con k-folder = 10 y 3 repticiones par obtner un estimado de lso limites de la metrica MAE. Esta metrica es el objetivo apra disminuir cuando se realize la implmentación del algorimo CLR.

In [25]:
# Se realiza la implmentacion del regresor lineal ridge
modelRidge = Ridge(alpha=0.59)
# fit model
modelRidge.fit(X_train, y_train)
# make a prediction
yhat = modelRidge.predict(X_test)
# summarize prediction
print('Predicted:', yhat)

Predicted: [3863.1624727  4516.1511383  5882.51844033 6212.94360276 3560.30219289
 3171.12064513 4225.74973429 6232.02468324 3747.82300974 3951.05989701
 4761.73991651 7007.27242236 4762.38220682 2568.44456567 4437.50510865
 3649.95122897 3710.86036314 3510.37252134 2494.63400997 3713.74512937
 5314.50418667 4915.20525879 3796.32569107 2712.59288286 6644.77380136
 6146.46751031 4941.31251298 5409.60092131 4490.27358361 6065.90200611
 2019.93346175 3994.55609534 4424.65521875 1717.89342618 5771.24055252
 5837.0343563  4830.85522382 4504.44434557 4419.29140882 4614.22775209
 4634.98617764 4786.70968383 4211.88086925 5320.34005267 3051.26800595
 3641.09269002 5247.7382292  3891.32220399 5729.73602996 6314.20353024
 4621.19290824 4222.63595231 4820.33427128 4510.42228164 2895.37856184
 5989.41763726 5338.61627879 4716.91205797 4748.68814811 5148.74070641
 5952.7815894  4749.78482427 4295.58866579 5654.49907658 6662.21820585
 4291.77402266 3863.59103283 6103.2309873  4804.51509341 6560.0835

In [26]:
# Metricas de desempeño del modelo
# Calculo del error 
print("======================= Conjunto de test ==========================")
print("MSE Ridge: ", (mean_squared_error(y_test,yhat,squared=False)))
print("MAE Ridge: ", mean_absolute_error(y_test,yhat))
print("r^2 Ridge: ", r2_score(y_test, yhat))

======================= Conjunto de test ==========================
MSE Ridge:  912.1032947910741
MAE Ridge:  678.3375476921879
r^2 Ridge:  0.5712874773124546


### Tunning Ajuste de Hiperparametros modelo Ridge

In [27]:
# define model
modelRidgeCv = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)
# Se define la cuadricula o grilla | metrica de comparación MAE 
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X, Y)
# summarize
print('MAE Ridge: ', results.best_score_)
print('Best Config:', results.best_params_)

MAE Ridge:  -658.0801058353832
Best Config: {'alpha': 0.59}


### Conclusiones
-  Con el regresor lineal ridge se obtiene un ajuste el 57% que es levement mejor que el regresor lineal lasso
-  La unica diferencia con lasso es el tipo de regularizacion mientras lasso , hace que los coeficentes puedan ser 0s llevando a ser seleccion de atributos. Ridge castiga aquellos coeficientes que no aportan  pero nunca llegan aa ser cero.

# IMPLMENTACIÓN DEL ENFOQUE CLUSTERWISE

## FASE #1 : Distribución de los datos

In [28]:
# Variables de entrada
# Dataset
DataSet = df_final.copy()
# Minimo numero de registros que debe tener cada grupo aceptado  (190)
MinimoRegistros = 190
# Error promedio aceptado de los grupos , permite seleccionar los grupos que mas se ajusten al modelo de RL
# Valor obtenido del proceso de ajuste con CV utilizando el regresor ridge
ErrorPromedioAceptado = 658.08
# Valor de tolerancia al error c/d vez que se crea el siguiente grupo | puede ser un grilla de diferentes valores a tolerar
ErrorPromedioAdicional = 10
# Coeficiente de correlación minimo aceptado por c/d grupo
CoeficienteCorrelacionMinimo = 0.88
# Lista de grupos aceptados
GruposAceptados = []
# Modelos aceptados x c/d grupo con sus atributos y valores de correlación
modelosAceptados = []
# Registros huerfanos que no fueron aceptados en ningun grupo
huerfanos = []



In [29]:
DataSet


,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
0,40,5,63,68,60000,13,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,43,5,64,63,60000,15,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,44,5,59,66,60000,12,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,45,5,64,59,60000,12,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,46,5,63,60,60000,16,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,4378,7,47,84,70000,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,4380,6,49,79,61000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
797,4382,7,48,95,65000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [30]:
# Calcular modelo de regresion lineal para todo el datset.
# Antes de entrar a la funcion asegurase que se a realizo la respectiva transformacion, codificacion , normalización de la data.
def calcularModeloRegresionLineal(DataSet):
    # Division de los datos
    # Variables dependientes | independiente
    Y = DataSet.RDT_AJUSTADO
    X = DataSet.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1)
    # Se define el modelo de regresion con Ridge | o el mejor regresor seleccionado.
    # Se realiza la implmentacion del regresor lineal ridge (alfa = 0.58) como la mejor configuración
    modelRidge = Ridge(alpha=0.58)
    # fit model
    model = modelRidge.fit(X,Y)
    # Obtengo el ajuste del modelo
    r_2 = model.score(X,Y)
    #print(r_2)
    # Retorna del modelo y el coeficiente de determinacion del modelo
    return [model, r_2]


In [31]:
# Funcion para ordenar los datos deacuerdo al error de predicción
# Entrada: Dataset, modelo, errorPromedioAceptado
# Salida: Dataset ordenado
def OrdenarDatosPorErrorDePrediccion(Dataset, model, ErrorPromedioAceptado):
    # Se dividen los datos
    # X:  Variables independendientes
    # Y:  Variables dependientes.
    Y = Dataset.RDT_AJUSTADO
    X = Dataset.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1)
    # Se realizan las predicciones del modelo
    yhat = model.predict(X)
    Dataset["Yhat"] = pd.Series(yhat)
    # Calculo del error absoluto para ca/d registro
    Dataset["EA"] = abs(Dataset.RDT_AJUSTADO - Dataset.Yhat)
    # Se ordena los registros de [menos a mayor] , tomando referencia el valor absoluto
    DataSetOrdenado = Dataset.sort_values(by=['EA'],ascending=True)
    
    # Se retorna el datset ordenado
    return DataSetOrdenado
    

In [32]:
# FUncion que permite crear el grupo con las mejores intancias que cumplan con lso criterios de aceptacion.
# - Mejor error absoluto
# - Numero de registros aceptado x grupo
def CrearGrupo(DataSetOrdenado):
    RegistrosAceptados = []
    for x in range(len(DataSetOrdenado)):
        if DataSetOrdenado.iloc[x].EA <= ErrorPromedioAceptado:
            RegistrosAceptados.append(DataSetOrdenado.iloc[x])

        else:
            print("Entra aqui")
            break
    
    # Registros que conforman n-grupos
    # Cada Grupo va ser un sub-dataset 
    return RegistrosAceptados


def LimpiezaColumnasDataframe(DataFrameGrupo):
    # Se eliminan algunas variables del Grupo 
    varEliminar = ["Yhat","EA"]
    DataFrameGrupo = DataFrameGrupo.drop(varEliminar,axis=1)
    # Se retorna el grupo
    return DataFrameGrupo
    
    

In [33]:
def EliminarRegistrosGrupo(Grupo):
    # Borrar del datset original las observaciones seleccionadas en cada grupo
    indexEliminar = list(Grupo.index)
    #print("Longitud Dataset Original")
    df_nuevo = DataSet[DataSet.index.isin(indexEliminar)== False]
    # Se retorna el
    return df_nuevo
    

In [34]:
# Implementacion del Algoritmo.
while(len(DataSet) != 0):
    print(DataSet)
    # 1. Calcular modelo de regresion dataset completo.
    modelrr, cd_r = calcularModeloRegresionLineal(DataSet)
    # 2. Seleccion de las mejores intancias (Dataset Ordenado)
    DataSetOr = OrdenarDatosPorErrorDePrediccion(DataSet, modelrr, ErrorPromedioAceptado)
    #print(len(DataSetOr))
    # 3. Crear Grupo con las mejores instancias.
    Grupo = pd.DataFrame(CrearGrupo(DataSetOr))
    #print("Cantidad de registros por grupo: ",Grupo.shape)
    if (len(Grupo) > 7):
        # 4. Eliminar variables 
        Grupo = LimpiezaColumnasDataframe(Grupo)
        DataSet = LimpiezaColumnasDataframe(DataSet)
        #print("Cantidad registos aceptados: ", Grupo.shape)
        #print("Longitud Dataset Original 3: ", DataSet.shape)
        # 5. Calculamos modelo de regresion Lineal para el grupo
        ModeloGrupo, R_2 = calcularModeloRegresionLineal(Grupo)
        # 6. Se evalua las condiciones del Coef Determinacion
        if (R_2 > CoeficienteCorrelacionMinimo):
            # 7. Se agregan el grupo y el modelo a las respectivas listas
            GruposAceptados.append(Grupo)
            modelosAceptados.append(ModeloGrupo)
            # 8. Se Eliminan las instancias del df original para la proxima iteracion
            DataSet = EliminarRegistrosGrupo(Grupo)
            # 9. Se adiciona el error promedio aceptado.
            ErrorPromedioAceptado = ErrorPromedioAceptado + ErrorPromedioAdicional
    else:
        # Salir del bucle
        Huerfanos = DataSet
        print("Logitud Huerfanos: ", Huerfanos.shape)
        break

    


     ID_LOTE  DIAS_EN_EMERGER  DIAS_EN_EMERGER_A_FLORECER  \
0         40                5                          63   
1         43                5                          64   
2         44                5                          59   
3         45                5                          64   
4         46                5                          63   
..       ...              ...                         ...   
794     4378                7                          47   
795     4379                5                          47   
796     4380                6                          49   
797     4382                7                          48   
798     4432                4                          49   

     DIAS_EN_FLORECER_A_COSECHAR  POBLACION_20DIAS_AJT  ALTURA_LOT  \
0                             68                 60000          13   
1                             63                 60000          15   
2                             66                 60000   

Entra aqui
     ID_LOTE  DIAS_EN_EMERGER  DIAS_EN_EMERGER_A_FLORECER  \
10       271                6                          56   
18       470                5                          47   
21       478                5                          79   
23       480                5                          46   
25       485                5                          42   
..       ...              ...                         ...   
792     4376                6                          47   
793     4377                6                          50   
795     4379                5                          47   
796     4380                6                          49   
798     4432                4                          49   

     DIAS_EN_FLORECER_A_COSECHAR  POBLACION_20DIAS_AJT  ALTURA_LOT  \
10                            72                 60000          10   
18                            78                 60000          10   
21                            59              

In [35]:
len(GruposAceptados)

3

In [36]:
GruposAceptados[0]

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
5,47.0,6.0,72.0,55.0,60000.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
59,710.0,6.0,47.0,81.0,60000.0,8.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,2448.0,5.0,47.0,72.0,71000.0,55.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,4183.0,5.0,46.0,79.0,75000.0,11.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
751,4283.0,5.0,46.0,85.0,73000.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2057.0,4.0,47.0,80.0,60000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
687,4195.0,5.0,46.0,90.0,73000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,1936.0,4.0,80.0,46.0,62000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
208,2056.0,4.0,44.0,81.0,55000.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [37]:
DataSet

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
18,470,5,47,78,60000,10,0,0,0,0,...,0,0,0,0,0,0,1,0,2789.469624,931.460376
21,478,5,79,59,68000,6,0,0,0,0,...,0,0,0,0,0,0,0,0,3503.291816,1119.571816
23,480,5,46,92,68000,4,0,0,0,0,...,1,0,0,0,0,0,0,0,4836.069063,2423.279063
25,485,5,42,71,65000,10,0,0,0,0,...,1,0,0,0,0,0,1,0,1674.536746,3664.993254
32,666,7,41,79,62000,10,0,0,0,0,...,0,0,0,0,0,0,1,0,2773.930684,624.909316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,4376,6,47,81,72000,16,0,0,0,0,...,1,0,0,0,0,0,0,0,NaN,NaN
793,4377,6,50,81,70000,13,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
796,4380,6,49,79,61000,17,0,0,0,0,...,1,0,0,0,0,0,0,0,NaN,NaN


In [38]:
modelrr, r_2 = calcularModeloRegresionLineal(DataSet)
##DatasetO = OrdenarDatosPorErrorDePrediccion(df_final, modelrr,ErrorPromedioAceptado)


ValueError: Input X contains NaN.
Ridge does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
DatasetO = OrdenarDatosPorErrorDePrediccion(df_final, modelrr,ErrorPromedioAceptado)

In [ ]:
DatasetO

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
5,47,6,72,55,60000,19,0,0,0,0,...,0,0,0,0,1,0,1,0,4957.923678,0.216322
59,710,6,47,81,60000,8,0,0,0,0,...,0,0,0,0,0,0,0,0,3711.126631,0.503369
309,2448,5,47,72,71000,55,0,0,0,0,...,0,0,0,0,0,0,0,0,4650.181438,0.978562
677,4183,5,46,79,75000,11,2,0,0,0,...,1,0,0,0,0,0,0,0,6047.733453,1.223453
751,4283,5,46,85,73000,5,0,0,0,0,...,1,0,0,0,1,0,0,0,5513.470924,1.840924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,3631,4,46,85,65000,5,0,0,0,0,...,0,0,0,0,1,0,1,0,4860.883986,2223.673986
170,1985,5,46,76,75000,7,0,0,0,0,...,0,0,0,0,0,0,1,0,3800.305804,2511.784196
462,2950,4,47,87,55000,4,0,0,0,0,...,0,0,0,0,1,0,1,0,3691.990717,2533.850717
360,2612,6,40,82,72000,18,0,0,0,0,...,1,0,0,0,0,0,1,0,4233.384715,2558.964715


In [ ]:
RegistroAceptadosGrupo = CrearGrupo(DatasetO)
RegistroAceptadosGrupo

Entra aqui


[ID_LOTE                           47.000000
 DIAS_EN_EMERGER                    6.000000
 DIAS_EN_EMERGER_A_FLORECER        72.000000
 DIAS_EN_FLORECER_A_COSECHAR       55.000000
 POBLACION_20DIAS_AJT           60000.000000
                                    ...     
 d.interno_nan                      0.000000
 drenaje_externo_NINGUNO            1.000000
 drenaje_externo_nan                0.000000
 Yhat                            4957.923678
 EA                                 0.216322
 Name: 5, Length: 191, dtype: float64,
 ID_LOTE                          710.000000
 DIAS_EN_EMERGER                    6.000000
 DIAS_EN_EMERGER_A_FLORECER        47.000000
 DIAS_EN_FLORECER_A_COSECHAR       81.000000
 POBLACION_20DIAS_AJT           60000.000000
                                    ...     
 d.interno_nan                      0.000000
 drenaje_externo_NINGUNO            0.000000
 drenaje_externo_nan                0.000000
 Yhat                            3711.126631
 EA             

In [ ]:
Grupo = pd.DataFrame(RegistroAceptadosGrupo)
# Grupo limpio
Grupo

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
5,47.0,6.0,72.0,55.0,60000.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,4957.923678,0.216322
59,710.0,6.0,47.0,81.0,60000.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3711.126631,0.503369
309,2448.0,5.0,47.0,72.0,71000.0,55.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4650.181438,0.978562
677,4183.0,5.0,46.0,79.0,75000.0,11.0,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6047.733453,1.223453
751,4283.0,5.0,46.0,85.0,73000.0,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5513.470924,1.840924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2057.0,4.0,47.0,80.0,60000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2554.806404,647.523596
687,4195.0,5.0,46.0,90.0,73000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6111.632785,648.842785
146,1936.0,4.0,80.0,46.0,62000.0,4.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3663.917228,649.967228
208,2056.0,4.0,44.0,81.0,55000.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2655.997765,653.667765


In [ ]:
GrupoLimpio = LimpiezaColumnasDataframe(Grupo)
GrupoLimpio

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
5,47.0,6.0,72.0,55.0,60000.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
59,710.0,6.0,47.0,81.0,60000.0,8.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,2448.0,5.0,47.0,72.0,71000.0,55.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,4183.0,5.0,46.0,79.0,75000.0,11.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
751,4283.0,5.0,46.0,85.0,73000.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2057.0,4.0,47.0,80.0,60000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
687,4195.0,5.0,46.0,90.0,73000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,1936.0,4.0,80.0,46.0,62000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
208,2056.0,4.0,44.0,81.0,55000.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
# Se calculan el modelo y el coeficiente de determinacion para cada Grupo
modelG, r_22 = calcularModeloRegresionLineal(GrupoLimpio)


In [ ]:
print("Coeficiente de determinacion del modelo ", r_22)

Coeficiente de determinacion del modelo  0.9338994938198788


In [ ]:
# Adicionamos los respectivos grupos y modelos
GruposAceptados.append(GrupoLimpio)
modelosAceptados.append(modelG)

In [ ]:
nuevo_def = EliminarRegistrosGrupo(GrupoLimpio)

In [ ]:
nuevo_def

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
10,271,6,56,72,60000,10,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
18,470,5,47,78,60000,10,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
21,478,5,79,59,68000,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,480,5,46,92,68000,4,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
25,485,5,42,71,65000,10,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,4376,6,47,81,72000,16,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
793,4377,6,50,81,70000,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,4380,6,49,79,61000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
modelosAceptados[2]

Ridge(alpha=0.58)

In [ ]:
X

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
0,5,63,68,60000,13,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,5,64,63,60000,15,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,5,59,66,60000,12,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,5,64,59,60000,12,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,5,63,60,60000,16,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,7,47,84,70000,18,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
795,5,47,81,62000,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,6,49,79,61000,17,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
797,7,48,95,65000,17,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Se realizan las prediccion del modelo
yhat = modelRidge.predict(X)
print(yhat)
print("Longitud de la lista de estimaciones : ", len(yhat))

[4915.20525879 4426.69736633 4895.71789756 4779.63076557 4834.1992899
 4699.76018796 4997.70302213 5604.85339539 4858.34993463 4477.17900837
 5032.12495143 4695.84034193 4820.33427128 6170.69469881 4495.32746878
 5320.34005267 4816.59191531 3955.84468034 4698.481142   3916.24323005
 4222.63595231 3793.73105337 4001.78466152 3236.82473121 4362.50403348
 4396.53653197 4248.58901155 4695.60825446 4236.87368032 3505.37496373
 3302.64780284 4291.77402266 4432.68022808 3861.43700523 3543.90938739
 3207.00854044 4621.72607541 5011.86618225 6839.25197343 6181.47540896
 5798.91727946 5032.47591846 5555.79460497 5488.10261575 4463.94131993
 3644.01121135 5068.49268075 3955.49467561 4315.02142557 5411.07567882
 4220.58300386 4238.85326097 4662.6700676  4148.60274533 4966.02113107
 4137.3842532  4194.12945938 3935.26197961 4629.65661542 3848.34504076
 3472.51901793 4393.14845878 4715.40633101 5072.29953183 5247.7382292
 3935.01026507 4291.96817771 4261.56491273 4451.39749678 4395.86404531
 3802.40

In [ ]:
# Se añade las predicciones a cada registro del datset
DataSet["Yhat"] = pd.Series(yhat)
# Calculo del error absoluto para ca/d registro
DataSet["EA"] = abs(DataSet.RDT_AJUSTADO - DataSet.Yhat)

In [ ]:
DataSet

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
0,40,5,63,68,60000,13,0,0,0,0,...,0,0,0,0,1,0,1,0,4915.205259,147.765259
1,43,5,64,63,60000,15,0,0,0,0,...,0,0,0,0,1,0,1,0,4426.697366,224.462634
2,44,5,59,66,60000,12,0,0,0,0,...,0,0,0,0,1,0,1,0,4895.717898,284.512102
3,45,5,64,59,60000,12,0,0,0,0,...,0,0,0,0,1,0,1,0,4779.630766,118.039234
4,46,5,63,60,60000,16,0,0,0,0,...,0,0,0,0,1,0,1,0,4834.199290,468.130710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,4378,7,47,84,70000,18,0,0,0,0,...,1,0,0,0,0,0,0,0,6728.663121,310.063121
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,6440.106461,858.706461
796,4380,6,49,79,61000,17,0,0,0,0,...,1,0,0,0,0,0,0,0,5897.104814,790.124814
797,4382,7,48,95,65000,17,0,0,0,0,...,1,0,0,0,0,0,1,0,5626.073801,138.116199


In [ ]:
# Se ordena los registros de menos a mayor , tomando referencia el valor absoluto
DataSetOrdenado = DataSet.sort_values(by=['EA'],ascending=True)

In [ ]:
DataSetOrdenado

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
677,4183,5,46,79,75000,11,2,0,0,0,...,1,0,0,0,0,0,0,0,6047.289748,0.779748
496,3045,5,46,90,72000,5,0,0,0,0,...,0,0,0,0,0,0,0,0,3859.862423,1.767577
721,4249,5,46,84,73000,6,0,0,0,0,...,1,0,0,0,0,0,0,0,5307.497247,5.167247
735,4267,5,48,66,58600,18,0,0,0,0,...,0,0,0,0,0,0,1,0,6022.269044,5.640956
30,654,4,44,79,60000,12,0,1,0,0,...,1,0,0,0,0,0,1,0,3302.647803,6.137803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2612,6,40,82,72000,18,0,0,0,0,...,1,0,0,0,0,0,1,0,4069.241217,2394.821217
84,1803,6,53,66,68000,11,0,0,0,0,...,0,0,0,0,1,0,1,0,1240.603825,2617.536175
462,2950,4,47,87,55000,4,0,0,0,0,...,0,0,0,0,1,0,1,0,3938.785746,2780.645746
482,3020,5,47,84,30000,55,0,0,0,0,...,1,0,0,0,0,0,0,0,4743.127858,2984.987858


In [ ]:
# Se crea el primer grupo |  con el numero de instancias seleccionadas

DataSetOrdenado[0:500]

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
677,4183,5,46,79,75000,11,2,0,0,0,...,1,0,0,0,0,0,0,0,6047.289748,0.779748
496,3045,5,46,90,72000,5,0,0,0,0,...,0,0,0,0,0,0,0,0,3859.862423,1.767577
721,4249,5,46,84,73000,6,0,0,0,0,...,1,0,0,0,0,0,0,0,5307.497247,5.167247
735,4267,5,48,66,58600,18,0,0,0,0,...,0,0,0,0,0,0,1,0,6022.269044,5.640956
30,654,4,44,79,60000,12,0,1,0,0,...,1,0,0,0,0,0,1,0,3302.647803,6.137803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,4314,5,48,85,60800,17,0,0,0,0,...,0,0,0,0,1,0,1,0,5992.579182,570.210818
777,4324,5,52,80,70000,18,0,0,0,0,...,0,0,0,0,1,0,0,0,6103.888849,570.531151
6,51,5,47,86,60000,14,0,0,0,0,...,0,0,0,0,1,0,1,0,4997.703022,570.963022
679,4187,5,46,90,73000,7,0,0,0,0,...,0,0,0,0,0,0,0,0,5751.986913,571.756913


In [ ]:
print("Longitud Dataset Ordenado", len(DataSetOrdenado))
DataSetOrdenado.iloc[0]

Longitud Dataset Ordenado 799


ID_LOTE                         4183.000000
DIAS_EN_EMERGER                    5.000000
DIAS_EN_EMERGER_A_FLORECER        46.000000
DIAS_EN_FLORECER_A_COSECHAR       79.000000
POBLACION_20DIAS_AJT           75000.000000
                                   ...     
d.interno_nan                      0.000000
drenaje_externo_NINGUNO            0.000000
drenaje_externo_nan                0.000000
Yhat                            6047.289748
EA                                 0.779748
Name: 677, Length: 191, dtype: float64

In [ ]:
DataSetOrdenado

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
677,4183,5,46,79,75000,11,2,0,0,0,...,1,0,0,0,0,0,0,0,6047.289748,0.779748
496,3045,5,46,90,72000,5,0,0,0,0,...,0,0,0,0,0,0,0,0,3859.862423,1.767577
721,4249,5,46,84,73000,6,0,0,0,0,...,1,0,0,0,0,0,0,0,5307.497247,5.167247
735,4267,5,48,66,58600,18,0,0,0,0,...,0,0,0,0,0,0,1,0,6022.269044,5.640956
30,654,4,44,79,60000,12,0,1,0,0,...,1,0,0,0,0,0,1,0,3302.647803,6.137803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2612,6,40,82,72000,18,0,0,0,0,...,1,0,0,0,0,0,1,0,4069.241217,2394.821217
84,1803,6,53,66,68000,11,0,0,0,0,...,0,0,0,0,1,0,1,0,1240.603825,2617.536175
462,2950,4,47,87,55000,4,0,0,0,0,...,0,0,0,0,1,0,1,0,3938.785746,2780.645746
482,3020,5,47,84,30000,55,0,0,0,0,...,1,0,0,0,0,0,0,0,4743.127858,2984.987858


In [ ]:
DataSetOrdenado.loc[0]

ID_LOTE                           40.000000
DIAS_EN_EMERGER                    5.000000
DIAS_EN_EMERGER_A_FLORECER        63.000000
DIAS_EN_FLORECER_A_COSECHAR       68.000000
POBLACION_20DIAS_AJT           60000.000000
                                   ...     
d.interno_nan                      0.000000
drenaje_externo_NINGUNO            1.000000
drenaje_externo_nan                0.000000
Yhat                            4915.205259
EA                               147.765259
Name: 0, Length: 191, dtype: float64

In [ ]:
GruposAceptados = []
for x in range(len(DataSetOrdenado)):
    #print(DataSetOrdenado.iloc[x])
    if DataSetOrdenado.iloc[x].EA <= ErrorPromedioAceptado:
        GruposAceptados.append(DataSetOrdenado.iloc[x])
    else:
        break



In [ ]:
len(GruposAceptados)

549

In [ ]:
Grupo = pd.DataFrame(GruposAceptados)
Grupo

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
677,4183.0,5.0,46.0,79.0,75000.0,11.0,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6047.289748,0.779748
496,3045.0,5.0,46.0,90.0,72000.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3859.862423,1.767577
721,4249.0,5.0,46.0,84.0,73000.0,6.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5307.497247,5.167247
735,4267.0,5.0,48.0,66.0,58600.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6022.269044,5.640956
30,654.0,4.0,44.0,79.0,60000.0,12.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3302.647803,6.137803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1902.0,4.0,55.0,65.0,75000.0,7.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3049.036510,185.193490
140,1927.0,5.0,54.0,63.0,62000.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2168.809175,185.840825
7,268.0,5.0,42.0,89.0,60000.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5604.853395,185.846605
754,4286.0,4.0,46.0,85.0,65000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5208.128915,187.221085


In [ ]:
# Se eliminan algunas variables del Grupo 
varEliminar = ['ID_LOTE',"Yhat","EA"]
Grupo.drop(varEliminar,axis=1)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
677,5.0,46.0,79.0,75000.0,11.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,5.0,46.0,90.0,72000.0,5.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
721,5.0,46.0,84.0,73000.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
735,5.0,48.0,66.0,58600.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
30,4.0,44.0,79.0,60000.0,12.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,4.0,55.0,65.0,75000.0,7.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
140,5.0,54.0,63.0,62000.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
7,5.0,42.0,89.0,60000.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
754,4.0,46.0,85.0,65000.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Divido los datos [Entrenamiento y pruebas]
Y_grupo = Grupo.RDT_AJUSTADO
X_grupo = Grupo.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1)

In [ ]:
# Calculamos el coeficiente de correlación del grupo.
modelo, r_2 = calcularModeloRegresionLineal(Grupo)
# Coeficente de correlacion
print("Coeficiente de determinación: ", r_2)
# Se realizan las predicciones con el modelo planteado
yhat_Grupo1 = modelo.predict(X_grupo)
# Calculo en coeficiente de determinacion con las predicciones
r2_s2 = r2_score(Y_grupo, yhat_Grupo1)
print("Coeficiente de determinacion con las predicciones: ", r2_s2)



Coeficiente de determinación:  0.9958146349895903
Coeficiente de determinacion con las predicciones:  0.9958146349895903


In [ ]:
# Adicionamos el df a una lista 
Grupos = []
Grupos.append(Grupo)

In [ ]:
# Borrar del datset original las observaciones seleccionadas en cada grupo
indexEliminar = list(Grupo.index)

In [ ]:
indexEliminar

[677,
 496,
 721,
 735,
 30,
 131,
 588,
 485,
 254,
 70,
 37,
 636,
 236,
 89,
 585,
 423,
 541,
 479,
 335,
 517,
 384,
 319,
 365,
 310,
 301,
 387,
 550,
 178,
 604,
 745,
 391,
 279,
 420,
 401,
 788,
 226,
 216,
 509,
 154,
 188,
 663,
 144,
 734,
 467,
 322,
 346,
 381,
 380,
 371,
 546,
 446,
 217,
 623,
 375,
 397,
 110,
 624,
 340,
 523,
 202,
 503,
 437,
 765,
 234,
 630,
 750,
 262,
 285,
 789,
 682,
 524,
 419,
 48,
 753,
 471,
 45,
 221,
 514,
 751,
 328,
 480,
 137,
 678,
 586,
 80,
 390,
 312,
 690,
 349,
 501,
 681,
 631,
 504,
 513,
 367,
 598,
 685,
 758,
 583,
 324,
 686,
 169,
 639,
 575,
 481,
 606,
 164,
 308,
 760,
 602,
 664,
 622,
 336,
 162,
 739,
 720,
 737,
 263,
 3,
 228,
 413,
 107,
 536,
 158,
 393,
 102,
 770,
 628,
 27,
 552,
 382,
 174,
 730,
 396,
 59,
 619,
 198,
 797,
 265,
 62,
 15,
 75,
 230,
 225,
 0,
 378,
 19,
 757,
 543,
 337,
 392,
 247,
 449,
 683,
 566,
 268,
 214,
 439,
 704,
 625,
 521,
 515,
 580,
 203,
 450,
 508,
 238,
 49,
 722,
 728

In [ ]:
# Filtro el nuevo dataset unicamente con los registros sobrantes que no esten en el grupo 1
# Borrar del datset original las observaciones seleccionadas en cada grupo
#indexEliminar = list(Grupo.index)
df_nuevo = DataSet[DataSet.index.isin(indexEliminar)== False]

In [ ]:
df_nuevo

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan,Yhat,EA
1,43,5,64,63,60000,15,0,0,0,0,...,0,0,0,0,1,0,1,0,4426.697366,224.462634
2,44,5,59,66,60000,12,0,0,0,0,...,0,0,0,0,1,0,1,0,4895.717898,284.512102
4,46,5,63,60,60000,16,0,0,0,0,...,0,0,0,0,1,0,1,0,4834.199290,468.130710
5,47,6,72,55,60000,19,0,0,0,0,...,0,0,0,0,1,0,1,0,4699.760188,258.379812
6,51,5,47,86,60000,14,0,0,0,0,...,0,0,0,0,1,0,1,0,4997.703022,570.963022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,4377,6,50,81,70000,13,0,0,0,0,...,0,0,0,0,0,0,0,0,6510.941334,553.008666
794,4378,7,47,84,70000,18,0,0,0,0,...,1,0,0,0,0,0,0,0,6728.663121,310.063121
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,6440.106461,858.706461
796,4380,6,49,79,61000,17,0,0,0,0,...,1,0,0,0,0,0,0,0,5897.104814,790.124814


<te>